In [ ]:
# code modeled after https://github.com/madelinehayes/seabirdNET
# note: code is almost identical to Harbor_Seals_Run_Model_Full and Harbor_Seals_Run_Model_Beach

In [ ]:
# train model
%run "../keras-retinanet/keras_retinanet/bin/train.py" --weights ../keras-retinanet/Models/resnet50_coco_best_v2.1.0.h5 \
--batch-size 2 --steps 280 --epochs 18 \
--snapshot-path ../Data/Elephant_Snapshot \
--random-transform \
--config elephant_config.ini \
csv ../Images/Elephant_Seals_Beach_Only/elephant_train_annotations.csv ../Images/Elephant_Seals_Beach_Only/elephant_classes.csv \
--val-annotations ../Images/Elephant_Seals_Full_Data/elephant_val_annotations.csv

In [ ]:
# convert model stored at second arg and store in path (final arg)
%run "../keras-retinanet/keras_retinanet/bin/convert_model.py" --config elephant_config.ini ../Data/Elephant_Snapshot/resnet50_csv_17.h5 ../Data/final_model_elephant_seal.h5

In [ ]:
# evaluate the model on the test set
%run "../keras-retinanet/keras_retinanet/bin/evaluate.py" csv ../Images/Elephant_Seals_Beach_Only/elephant_test_annotations.csv ../Data/elephant_seal_correct_annotations/elephant_classes.csv ../Data/final_model_elephant_seal.h5

In [ ]:
%matplotlib inline

%reload_ext autoreload
%autoreload 2

import keras

from keras_retinanet.models import retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color


import matplotlib.pyplot as plt
import cv2
import sys
import os
import numpy as np
import time
import json
from random import shuffle

import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(get_session())

In [ ]:
sys.path.insert(0, '../keras-retinanet/keras_retinanet')
from utils.gpu import setup_gpu

# use this to change which GPU to use
gpu = '1'

# set the modified tf session as backend in keras
setup_gpu(gpu)

In [ ]:
model_path = '../Data/final_model_elephant_seal17_best.h5'

#print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')

#print(model.summary())

#make sure this aligns with your annotations if you didn't use mine
labels_to_names = {"0":"cow","1":"bull","2":"pup"}

In [ ]:
image_dir = "../Images/Elephant_Seals_Beach_Only/"

image_list = []
for root, dirs, files in os.walk(image_dir):
    for filename in files:
        if filename.lower().endswith(('.png')):
            image_list.append(image_dir + filename)
print(len(image_list))

In [ ]:
# visualize 10 random images
visualize = True
min_score = 0.5

detections = {}

total_time = 0

count = 0
detection_iterations = 10
shuffle(image_list)

for image_path in image_list: 
    if count > detection_iterations:
        break
    else: 
        count +=1
    
    image = read_image_bgr(image_path)
    
    if visualize:
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start 
    
    boxes /= scale
    if any(score >= min_score for score in scores [0]):
        detections[image_path] = []
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
        if visualize: 
            color = label_color(label)
            
            draw_box(draw, b, color=color)
            
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            
    if any(score >= min_score for score in scores[0]):
        if visualize:
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
            
print("Finished, time per image:", total_time/len(image_list))

In [ ]:
# run through all the images
min_score = 0.5

detections = {}

cows = 0
pups = 0
bulls = 0

total_time = 0

for image_path in image_list:
    
    image = read_image_bgr(image_path)
          
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start
    
    boxes /= scale  
        
    if any(score >= min_score for score in scores[0]):
        detections[image_path] = []
        
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)    
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        if label == 0:
            cows += 1
        elif label == 1:
            bulls += 1
        elif label == 2:
            pups += 1
        
            
print("Finished, time per image:", total_time/len(image_list))

In [ ]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [ ]:
# write detections
with open('../Test_Output/detections.json', 'w') as fp:
    json.dump(detections, fp, cls=MyEncoder)

In [ ]:
print(len(detections))
print("cows:", cows)
print("bulls:", bulls)
print("pups:", pups)